In [19]:
import argparse
import math
import cmath
import sys
import time
import copy

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization
from keras import regularizers
from keras.layers.noise import GaussianNoise
from keras.layers import Conv1D, MaxPooling1D
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils.np_utils import to_categorical
##K.set_image_dim_ordering('th')
K.set_image_data_format('channels_first')
print(K.image_data_format())

## required for efficient GPU use
import tensorflow as tf
##from keras.backend import tensorflow_backend
from tensorflow.python.keras.backend import set_session 
config = tf.compat.v1.ConfigProto(gpu_options=tf.compat.v1.GPUOptions(allow_growth=True))
session = tf.compat.v1.Session(config=config)
set_session(session)
## required for efficient GPU use

import os
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra

channels_first


In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
## 数据填充
def fill_data(data_array, target_line, fill_mark):
  if (data_array.shape[0] >= target_line):
    return data_array[:target_line]
  elif (data_array.size > 0) is False:
    return np.zeros(shape=(10, 3))
  else:
    fill_array = np.zeros(shape=(target_line-data_array.shape[0], data_array.shape[1]))
    fill_array[:, :] = fill_mark
    return np.vstack((data_array, fill_array))

In [22]:
## 经纬度转化两点间距离
def distance(data_array):
  row = data_array.shape[0]
  distance_array = np.zeros((1, row))
  print("distance_array", distance_array)
  data_array = np.insert(data_array, 4, values = distance_array, axis = 1)
  print("data_array", data_array)
  for i in range(row):
    if i != (row-1):
      lat1 = math.radians(data_array[i][0])
      lon1 = math.radians(data_array[i][1])
      lat2 = math.radians(data_array[i+1][0])
      lon2 = math.radians(data_array[i+1][1])
      diff_lat = lat2 - lat1
      diff_lon = lon2 - lon1
      data_array[i][4] = 2 * math.asin(math.sqrt(math.sin(diff_lat/2.0) ** 2 + math.cos(lat1) * math.cos(lat2) * (math.sin(diff_lon/2.0) ** 2))) * 6378.137
  return data_array[:,2:5]


In [23]:
#结果输出连线
def draw_polylines(points, map, color):
    folium.PolyLine(points, color=color, weight=2.5, opacity=1).add_to(map)

In [24]:
class preprocess:
  from sklearn import model_selection as ms
  from sklearn import preprocessing as pp

  %cd /content/drive/MyDrive/Anomaly_Detection/Input_2
#-------------------------------------------------------------------------------
  real_ship_number = 30
  generation_ship_number = 15
  total_ship_number = real_ship_number + generation_ship_number
  real_ship_period = 140
  generation_period_number = np.load("../Input_2/Generation_data/Train_data/Number_period_list.npy").tolist()
  generation_period_number_test = np.load("../Input_2/Generation_data/Test_data/Number_period_list_test.npy").tolist()
  X_for_draw = []
#-------------------------------------------------------------------------------
  for shipnumber in range(1,total_ship_number+1):
    if shipnumber <= real_ship_number:
      period = real_ship_period 
      period_test = real_ship_period
    else:
      period = generation_period_number[shipnumber-real_ship_number-1]
      period_test = generation_period_number_test[shipnumber-real_ship_number-1]
#-------------------------------------------------------------------------------
    for i in range(0, period):
      #读数据
      if shipnumber <= real_ship_number:
        Input_train_file = ("../Input_2/Real_data/2019_01_01/segment_ship_%d_period_%d_2019_01_01_region_1.csv" %(shipnumber,i))
        if os.path.isfile(Input_train_file) is False:
          continue
        x_input_frame = pd.read_csv(Input_train_file) ## input data
        x_train = np.array(x_input_frame.values[:, 4:8].tolist())
      else:
        Input_train_file = ("../Input_2/Generation_data/Train_data/generation_data_fillzero_ship_%d_period_%d.csv" %(shipnumber-real_ship_number,i))
        x_input_frame = pd.read_csv(Input_train_file) ## input data
        x_train = np.array(x_input_frame.values[:, 1:].tolist())

      # 经纬度转化距离
      if x_train.shape[0] != 0:
        x_train = distance(x_train)

      ## 标准化
      ss = pp.StandardScaler() 
      if (x_train.size > 0):
        x_train = ss.fit_transform(x_train)

      ## 数据填充 
      x_train = fill_data(x_train, 10, 0)
      print(shipnumber,i)
      if shipnumber < 5:
        print("数据填充后数据\n", x_train)

      #label
      if shipnumber <= real_ship_number:
        y_train = np.array([0, 1])
      else:
        y_train = np.array([1, 0])

      if (i == 0 and shipnumber == 1):  
        X_train = np.reshape(x_train, (1, x_train.shape[0], x_train.shape[1])).tolist() ## reshape data， 多加上一维度
        Y_train = np.reshape(y_train, (1, y_train.shape[0])).tolist() ## reshape data， 多加上一维度
      else:
        #连接各个例子 3d 2d
        print(type(X_train))
        X_train.append(x_train.tolist())
        Y_train.append(y_train.tolist())
#-------------------------------------------------------------------------------
    for i in range(0, period_test):
      #读数据
      if shipnumber <= real_ship_number:
        Input_test_file = ("../Input_2/Real_data/2020_06_01/segment_ship_%d_period_%d_2020_06_01_region_1.csv" %(shipnumber,i))
        if os.path.isfile(Input_test_file) is False:
          continue
        x_test_input_frame = pd.read_csv(Input_test_file) ##input data
        x_test = np.array(x_test_input_frame.values[:, 4:8].tolist())
      else:
        Input_test_file = ("../Input_2/Generation_data/Test_data/generation_data_fillzero_test_ship_%d_period_%d.csv" %(shipnumber-real_ship_number,i))
        x_test_input_frame = pd.read_csv(Input_test_file) ##input data
        x_test = np.array(x_test_input_frame.values[:, 1:].tolist())

      #为输出结果在地图上准备的
      x_for_draw = x_test

      # 经纬度转化距离
      if x_test.shape[0] != 0:
        x_test = distance(x_test)
      
      # 标准化
      ss = pp.StandardScaler() 
      if (x_test.size > 0):
        x_test = ss.fit_transform(x_test)

      # 数据填充 
      x_test = fill_data(x_test, 10, 0)
      x_for_draw = fill_data(x_for_draw, 10, 0)
      print(shipnumber,i)
      if shipnumber < 5:
        print("数据填充后数据\n", x_test)

      #label
      if shipnumber <= real_ship_number:
        y_test = np.array([0, 1])
      else:
        y_test = np.array([1, 0])

      if (i == 0 and shipnumber == 1): 
        X_for_draw =  np.reshape(x_for_draw, (1, x_for_draw.shape[0], x_for_draw.shape[1])).tolist()
        X_test = np.reshape(x_test, (1, x_test.shape[0], x_test.shape[1])).tolist()
        Y_test = np.reshape(y_test, (1, y_test.shape[0])).tolist()
      else:
        #连接各个例子 3d 2d
        print(type(X_train))
        X_for_draw.append(x_for_draw.tolist())
        X_test.append(x_test.tolist())
        Y_test.append(y_test.tolist())
#-------------------------------------------------------------------------------
  #数据类型转换，输出格式 验证
  np.save("../Input_2/Generation_data/Train_data/X_train.npy", X_train)
  np.save("../Input_2/Generation_data/Train_data/Y_train.npy", Y_train)
  np.save("../Input_2/Generation_data/Test_data/X_test.npy", X_test)
  np.save("../Input_2/Generation_data/Test_data/Y_test.npy", Y_test)
  X_train = np.array(X_train)
  X_test = np.array(X_test)
  Y_train = np.array(Y_train)
  Y_test = np.array(Y_test)
      
  print('x_train shape: {}'.format(X_train.shape))
  print('y_train shape: {}'.format(Y_train.shape))
  print('x_test shape: {}'.format(X_test.shape))
  print('y_test shape: {}'.format(Y_test.shape))

流式输出内容被截断，只能显示最后 5000 行内容。
<class 'list'>
distance_array [[0. 0. 0. 0. 0. 0. 0. 0.]]
data_array [[ 4.9376480e+01 -1.2327147e+02  2.0000000e-01  1.5710000e+02
   0.0000000e+00]
 [ 4.9376450e+01 -1.2327147e+02  0.0000000e+00  2.4000000e+02
   0.0000000e+00]
 [ 4.9376450e+01 -1.2327147e+02  0.0000000e+00  2.9410000e+02
   0.0000000e+00]
 [ 4.9376450e+01 -1.2327147e+02  0.0000000e+00  2.2940000e+02
   0.0000000e+00]
 [ 4.9376460e+01 -1.2327147e+02  0.0000000e+00  3.0450000e+02
   0.0000000e+00]
 [ 4.9376460e+01 -1.2327147e+02  0.0000000e+00  3.0740000e+02
   0.0000000e+00]
 [ 4.9376460e+01 -1.2327147e+02  0.0000000e+00  5.8600000e+01
   0.0000000e+00]
 [ 4.9376370e+01 -1.2327155e+02  0.0000000e+00  1.5560000e+02
   0.0000000e+00]]
6 31
<class 'list'>
distance_array [[0. 0. 0. 0. 0. 0. 0. 0. 0.]]
data_array [[  49.37753 -123.27135    7.6        6.3        0.     ]
 [  49.38029 -123.27083    9.        10.4        0.     ]
 [  49.38281 -123.27126    8.9      335.2        0.     ]
 [  49.38477

+ 数据格式已经搞对，要可视化表达一下，模型对应参数要改，因为输入数据格式改了

1. 负样本的合并与打乱

  1.1 衍生数据的时间序列需要生成

2. 为了优化精确度，要考虑几个问题

  2.1 缺少的数据怎么处理 填充删除

  2.2 什么时候用Standardscaler最有效

  2.3 还有什么类似Standardscaler的 更适配这种情况的方法

+ 测试和训练数据要选择好并且分开，然后进模型跑一边看精确度

In [29]:
#size of parameters 设置参数
batch_size = 100
num_classes = 2
epochs = 500
filter_size=3
#noise = 1
droprate=0.50

In [30]:
#Start Neural Network
model = Sequential() ##  规定 model 只有一个输入和一个输出

#convolution 1st layer
model.add(Conv1D(64, kernel_size=(filter_size), padding="same",
                 activation='relu',
                 input_shape=(10, 3)))
model.add(BatchNormalization())
model.add(Dropout(droprate))
          
#convolution 2nd layer
model.add(Conv1D(128, kernel_size=(filter_size), activation='relu', padding="same"))
model.add(BatchNormalization())
model.add(MaxPooling1D(strides=1))
model.add(Dropout(droprate))

#convolution 3rd layer
model.add(Conv1D(256, kernel_size=(filter_size), activation='relu', padding="same"))
model.add(BatchNormalization())
model.add(MaxPooling1D(strides=1))
model.add(Dropout(droprate))

#FCN 1st layer
model.add(Flatten())
model.add(Dense(64,use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(droprate))

#FCN 2nd layer
model.add(Dense(32,use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(droprate))

#FCN 3rd layer
model.add(Dense(16,use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(droprate))

#FCN final layer
model.add(Dense(2))
model.add(Activation('softmax'))## 用来算出概率

model.compile(loss="binary_crossentropy", optimizer="Adam", metrics=['accuracy']) ##使用adam 优化器

model.summary() ##将 keras 结构可视化

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_9 (Conv1D)           (None, 10, 64)            640       
                                                                 
 batch_normalization_18 (Bat  (None, 10, 64)           256       
 chNormalization)                                                
                                                                 
 dropout_18 (Dropout)        (None, 10, 64)            0         
                                                                 
 conv1d_10 (Conv1D)          (None, 10, 128)           24704     
                                                                 
 batch_normalization_19 (Bat  (None, 10, 128)          512       
 chNormalization)                                                
                                                                 
 max_pooling1d_6 (MaxPooling  (None, 9, 128)          

In [31]:
!pip install keras_sequential_ascii
from keras_sequential_ascii import keras2ascii

keras2ascii(model)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
           OPERATION           DATA DIMENSIONS   WEIGHTS(N)   WEIGHTS(%)

               Input   #####     10    3
              Conv1D    \|/  -------------------       640     0.2%
                relu   #####     10   64
  BatchNormalization    μ|σ  -------------------       256     0.1%
                       #####     10   64
             Dropout    | || -------------------         0     0.0%
                       #####     10   64
              Conv1D    \|/  -------------------     24704     9.5%
                relu   #####     10  128
  BatchNormalization    μ|σ  -------------------       512     0.2%
                       #####     10  128
        MaxPooling1D   Y max -------------------         0     0.0%
                       #####      9  128
             Dropout    | || -------------------         0     0.0%
                       #####      9  128
              Conv1D   

In [32]:
#Save Model=ON
history = model.fit(preprocess.X_train, preprocess.Y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(preprocess.X_test, preprocess.Y_test),shuffle=True)

score = model.evaluate(preprocess.X_test, preprocess.Y_test, verbose=0)

#print loss and accuracy
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/500
5/5 [==============================] - 2s 78ms/step - loss: 0.8649 - accuracy: 0.4566 - val_loss: 0.6859 - val_accuracy: 0.6940
Epoch 2/500
5/5 [==============================] - 0s 21ms/step - loss: 0.8802 - accuracy: 0.4499 - val_loss: 0.6822 - val_accuracy: 0.7651
Epoch 3/500
5/5 [==============================] - 0s 18ms/step - loss: 0.8353 - accuracy: 0.4944 - val_loss: 0.6801 - val_accuracy: 0.7544
Epoch 4/500
5/5 [==============================] - 0s 15ms/step - loss: 0.8124 - accuracy: 0.4922 - val_loss: 0.6770 - val_accuracy: 0.7509
Epoch 5/500
5/5 [==============================] - 0s 15ms/step - loss: 0.8006 - accuracy: 0.5390 - val_loss: 0.6749 - val_accuracy: 0.7509
Epoch 6/500
5/5 [==============================] - 0s 19ms/step - loss: 0.7530 - accuracy: 0.5568 - val_loss: 0.6727 - val_accuracy: 0.7509
Epoch 7/500
5/5 [==============================] - 0s 16ms/step - loss: 0.7581 - accuracy: 0.5479 - val_loss: 0.6699 - val_accuracy: 0.7509
Epoch 8/500
5/5 [===

In [33]:
from sklearn.metrics import classification_report, confusion_matrix
import folium

#原始概率
y_pred = model.predict(preprocess.X_test)
print("原始概率",y_pred)

#或者，label12中取最大的
y_pred = np.argmax(y_pred, axis=1)
#print(y_pred)

#或者， 大于0.5的算1
##y_pred = np.argmax(model.predict(X_test),axis=1)
y_pred = (model.predict(preprocess.X_test) > 0.5).astype("int32")
#print(y_pred)

#或者， label1/2概率大于于0.01的才为1，label1为1整体为异常 label2为1整体正常
y_pred[:,0] = (model.predict(preprocess.X_test)[:,0] > 0.1).astype("int32")
y_pred[:,1] = (model.predict(preprocess.X_test)[:,1] > 0.9).astype("int32")

#-------------------------------------------------------------------------------
mapname_draw = "./Map/result_map/map_test.html"
latitude = preprocess.X_for_draw[0][0][0]
longitude = preprocess.X_for_draw[0][0][1]
ocean_map = folium.Map(location=[latitude, longitude], zoom_start=11)
for i in range(len(preprocess.X_for_draw)):
    lat_list = [x[0] for x in preprocess.X_for_draw[i]]
    lon_list = [x[1] for x in preprocess.X_for_draw[i]]
    location_clear = []
    incidents = folium.map.FeatureGroup()
    for lat, lon in zip(lat_list, lon_list):
      if lat != 0 and lon != 0:
        location_clear.append([lat, lon])
        incidents.add_child(
            folium.CircleMarker(
              [lat, lon],
              radius=0.05, # define how big you want the circle markers to be
              color='black',
              fill=True,
              fill_color='red',
              fill_opacity=0.4
            )
        )
    ocean_map.add_child(incidents)
    if len(location_clear) > 0:
      if y_pred[i,0] == 1 and preprocess.Y_test[i,0] == 1:
        draw_polylines(location_clear, ocean_map, "red")
      elif y_pred[i,0] == 1 and preprocess.Y_test[i,0] == 0:
        draw_polylines(location_clear, ocean_map, "yellow")
      elif y_pred[i,1] == 1 and preprocess.Y_test[i,1] == 1:
        draw_polylines(location_clear, ocean_map, "green")
      elif y_pred[i,1] == 1 and preprocess.Y_test[i,1] == 0:
        draw_polylines(location_clear, ocean_map, "blue")
ocean_map.save(mapname_draw)
#-------------------------------------------------------------------------------
np.set_printoptions(threshold=np.inf)
pd.set_option('display.width', 300) # 设置字符显示宽度
pd.set_option('display.max_rows', None) # 设置显示最大行
pd.set_option('display.max_columns', None) # 设置显示最大列，None为显示所有列
print("0.01threshold",y_pred)
#-------------------------------------------------------------------------------
#输出y_pred为二维数组
p=model.predict(preprocess.X_test)

target_names = ["Abnormal", "Normal"]
#print(classification_report(np.argmax(preprocess.Y_test,axis=1), y_pred, target_names=target_names, digits=4))
#print(confusion_matrix(np.argmax(preprocess.Y_test, axis=1), y_pred))
print(classification_report(preprocess.Y_test, y_pred, target_names=target_names, digits=4))
print(confusion_matrix(preprocess.Y_test[:,1], y_pred[:,1]))

9/9 [==============================] - 0s 3ms/step
原始概率 [[9.93843913e-01 6.15613954e-03]
 [9.58671809e-10 1.00000000e+00]
 [2.12347793e-08 1.00000000e+00]
 [3.32951693e-08 1.00000000e+00]
 [8.91129837e-09 1.00000000e+00]
 [1.96992778e-09 1.00000000e+00]
 [4.86198504e-09 1.00000000e+00]
 [2.77816685e-06 9.99997258e-01]
 [9.81015660e-08 9.99999881e-01]
 [3.88271859e-10 1.00000000e+00]
 [2.06848672e-09 1.00000000e+00]
 [1.07589074e-07 9.99999881e-01]
 [1.25284361e-08 1.00000000e+00]
 [7.59089946e-09 1.00000000e+00]
 [1.18442101e-08 1.00000000e+00]
 [4.06789225e-09 1.00000000e+00]
 [3.76921179e-08 1.00000000e+00]
 [1.57734590e-08 1.00000000e+00]
 [2.02973038e-11 1.00000000e+00]
 [8.16241019e-10 1.00000000e+00]
 [2.17688334e-09 1.00000000e+00]
 [1.73344861e-09 1.00000000e+00]
 [5.61641968e-08 1.00000000e+00]
 [5.24864570e-12 1.00000000e+00]
 [6.51178722e-10 1.00000000e+00]
 [2.81760837e-08 1.00000000e+00]
 [3.20622556e-11 1.00000000e+00]
 [1.58308666e-09 1.00000000e+00]
 [1.98696299e-07 9.9

SVM

In [ ]:
##训练时发现，将perd的数据和本来的数据作比较，然后看精度如何
##训练不同区域的数据
##让异常数据更精确
##因为是csv文件，所以训练起来gpu压力会小